## ArcGIS Online Data

https://www.arcgis.com/index.html

The ArcGIS Python API allows you to connect to your ArcGIS Online Organization through Python. Just like accessing ArcGIS Online through a web browser, you need to login to access your data.

Data in ArcGIS Online is stored as feature layers and tables. These have similar properties to geodatabases and feature classes. 

Featuresets are similar to the feature layers we have worked with in arcpy. These are used to update the feature services.

Link to arcgis Python API documentation: https://developers.arcgis.com/python/api-reference/

API stands for: application programming interface

The [GIS](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#gis) object is the way the Python API for ArcGIS handles your credentials to ArcGIS Online.  [This](https://developers.arcgis.com/python/guide/the-gis-module/) page gives a pretty decent overview of the different useful parts of a GIS object.

![Image of API](https://developers.arcgis.com/assets/img/python-graphics/guide_gis_module_01.png)

In [ ]:
import os
import arcgis
from IPython.display import display

ArcGIS package is not the same as the ArcPy Package!

In [ ]:
mygis = arcgis.GIS('pro')

In [ ]:
mygis.users.me

In [ ]:
#creating a GIS object outside of arcgis pro

#This will cause an error
private_gis = arcgis.GIS(URL, username, password)

### Creating a map within Jupyter Notebook

In [ ]:
map_sample = mygis.map("Walnut Creek, CA")
map_sample

https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html?highlight=content%20search#arcgis.gis.ContentManager.search

In [ ]:
items = mygis.content.search('Walnut Creek', item_type='feature layer', outside_org=True)

In [ ]:
items

In [ ]:
for item in items:
    display(item)

In [ ]:
cal_fire_polygon = items[0]
map_sample.add_layer(cal_fire_polygon)
map_sample.zoom_to_layer(cal_fire_polygon.layers[0])

In [ ]:
map_sample

### Uploading Data to ArcGIS Online

In [ ]:
item_properties = {"type": "Shapefile",
                   "title" : "Walnut Creek Restaurants",
                    "tags": "Restaurants, Walnut Creek, CA",
                   "description" : "Restaurants in Walnut Creek, CA. Created 2022 from Python",
                   "commentsEnabled" : False}

project_path = os.getcwd()
wc_restaurants_full_path = os.path.join(project_path, 'wc_restaurants.zip')

The .add method will just upload the shapefile or gdb to arcgis online. It will not publish it as a feature layer

In [ ]:
wc_restaurants_item_shp = mygis.content.add(item_properties, data=wc_restaurants_full_path)

To Publish it as a feature layer that we can then use with the API, we must use the .publish method

In [ ]:
wc_restaurants_feature_layer = wc_restaurants_item_shp.publish()

In [ ]:
map_sample.add_layer(wc_restaurants_feature_layer)
map_sample

### Great! Now we have the Walnut Creek layer uploaded as feature layer on AGOL

#### We can now delete the shapefile that we uploaded and we used for publishing

In [ ]:
wc_restaurants_item_shp.delete()

### Accessing and Editing the data

In order to view, query, and edit the data that's on AGOL, a feature layer object within python must be made.

These can be created either using the search method, or using the rest service URL.

After the feature layer is created, the layer can be queryed to create a feature set. 
<br>A feature set enables the user to make changes
and then set these changes as permanent back in the original feature layer

It is good to try to understand the differences between a feature set and a feature layer

In [ ]:
#to create a feature layer object, we use:
arcgis.features.FeatureLayer("item object or URL goes here!")

In [ ]:
# we can search for the data

local_items = mygis.content.search('Walnut Creek Restaurants', item_type='feature layer', outside_org=False)

for i in local_items:
    display(i)

In [ ]:
#Then to get the above item as a feature layer object within python, we would:

arcgis.features.FeatureLayer(local_items[0])

In [ ]:
# Or we could do the following (what I always recommend)

#each rest service url is different, make sure to replace this path with yours before moving on!
Walnut_Creek_Restaurants_rest_url = r'https://services.arcgis.com/JbibFpsuaEQr3FFG/ArcGIS/rest/services/wc_restaurants/FeatureServer/0'
arcgis.features.FeatureLayer(Walnut_Creek_Restaurants_rest_url)

#### to create a Feature Set, a Feature Layer must be queried upon. This can be an empty query if you would like to return all records

In [ ]:
feature_layer = arcgis.features.FeatureLayer(Walnut_Creek_Restaurants_rest_url)

feature_layer.query()

In [ ]:
for row in feature_layer.query():
    print(row)

In [ ]:
row

In [ ]:
row.attributes['name']

In [ ]:
# query has a lot of parameter options
feature_layer.query(return_count_only=True)

In [ ]:
feature_layer.query(where= "cuisine = 'mexican'", return_count_only=True)

In [ ]:
#This is really handy, but not very practical if we are trying to update the data we already have.
#This is because it returns a dataframe, and we can't turn it back into a feature layer very easily.
test_df = feature_layer.query(as_df=True)

test_df.head(5)

In [ ]:
#however, we can save it as a featureclass:
example = os.path.join(arcpy.env.workspace, 'test_layer')
test_df.spatial.to_featureclass(example, sanitize_columns=False)

#### The above method of querying a rest service and returning it as a dataframe, and then creating a local feature class from the dataframe, <br> can be useful for the following reasons:

##### 1. Backing up data from AGOL on a routine basis
##### 2. Creating a report on data that currently exists in AGOL

## Editing AGOL Data

### We are able to edit our data in AGOL in two ways:

### Using a feature set

### And using a calculate fields call onto the feature layer

Feature sets are comparable to cursors in arcpy!

In [ ]:
# First lets try using a feature set to replace all blank cuisine's with a value

feature_set = feature_layer.query()

for row in feature_set:
    cuisine_value = row.attributes['cuisine']
    if cuisine_value == " ":
        print(row)

In [ ]:
for row in feature_set:
    cuisine_value = row.attributes['cuisine']
    if cuisine_value == " ":
        row.attributes['cuisine'] = 'missing'

#update feature layer
feature_layer.edit_features(updates=feature_set)

In [ ]:
feature_layer.query(as_df=True).head(20)

In [ ]:
# For less complex changes, we can make changes directly on the feature layer, using a calculate fields method:

#in the below example, lets pretend that we need to change the rating of the first 9 records to a value of 1

feature_layer.calculate(where="FID < 10",
                   calc_expression={"field": "Rating", "value" : "1"})

In [ ]:
feature_layer.query(as_df=True).head(20)